#Install


In [1]:
! pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [2]:
! pip install --upgrade sendgrid

     |████████████████████████████████| 81kB 2.1MB/s 
  Created wheel for python-http-client: filename=python_http_client-3.3.1-cp36-none-any.whl size=7523 sha256=f839825c077a1a78de28ed1a62435c097b0776355f59d40c3cb0282a66fabb8a
  Stored in directory: /root/.cache/pip/wheels/14/67/9b/abeb3c15c0d92ad24c3a17474cb833d79a92b61f607320aa05
  Created wheel for starkbank-ecdsa: filename=starkbank_ecdsa-1.1.0-cp36-none-any.whl size=13646 sha256=f5ab1e50ccd3bec10df9d0b86469b94e9790f1608940d7f33a4d4d354429012f
  Stored in directory: /root/.cache/pip/wheels/15/a8/1b/b0d920b8182516ed69179b3ed82e46ce90f4de20d0ccb4b02d
Successfully built python-http-client starkbank-ecdsa


In [3]:
! pip install --upgrade premailer

     |████████████████████████████████| 409kB 3.9MB/s 


#ENV
Two variables SEND_GRID and KEY_IMGBB are required, you can do this at:

- https://sendgrid.com/
- https://imgbb.com/

In [4]:
from env import * # !Just Bruno Nascimento

#Imports

In [5]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import random
import string
from PIL import ImageFont, ImageDraw, Image
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import base64
import requests
import json
from datetime import datetime
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
from premailer import transform

#Google Credentials


In [6]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Variables

In [7]:
drive_directory = '/content/drive/My Drive/action_email_with_image/images/'

#Utils


##Email

In [8]:
def sendemail(from_addr, to_addr_list,  subject, message):
    # Create the body of the message (a plain-text and an HTML version).
    html_finaly = transform(message)

    email_message = Mail(
        from_email=from_addr,
        to_emails=to_addr_list,
        subject=subject,
        html_content=html_finaly
    )
    try:

        sg = SendGridAPIClient(SEND_GRID)
        response = sg.send(email_message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as error:
        print(error)

##Wrap text

In [9]:
def wrap_text(text, width, font):
    text_lines = []
    text_line = []
    text = text.replace('\n', ' [br] ')
    words = text.split()
    font_size = font.getsize(text)

    for word in words:
        if word == '[br]':
            text_lines.append(' '.join(text_line))
            text_line = []
            continue
        text_line.append(word)
        w, h = font.getsize(' '.join(text_line))
        if w > width:
            text_line.pop()
            text_lines.append(' '.join(text_line))
            text_line = [word]

    if len(text_line) > 0:
        text_lines.append(' '.join(text_line))

    return '\n'.join(text_lines),len(text_lines)

##Random char

In [10]:
def random_char(y):
    return ''.join(random.choice(string.ascii_letters) for x in range(y))

##Create image

In [87]:
def create_img(type_img,text):
    image = cv2.imread(f'{drive_directory}model/{type_img}.png')
    

    # Convert to PIL Image
    cv2_im_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(cv2_im_rgb)

    draw = ImageDraw.Draw(pil_im)

    # use a bitmap font
    font = ImageFont.truetype(font='/content/drive/My Drive/action_email_with_image/fonts/arial.ttf', size=11, index=0)

    # Draw the text
    text,num_lines = wrap_text(text, 255, font)

    # Remove blank lines if less than 10 lines
    begin_line = 140 if num_lines <=10  else 95

    # Add text
    draw.text((310, begin_line), text, font=font,fill=(27, 54, 93))

    name_file = f'output_{str(random.randrange(1000)).zfill(4)}_{random_char(10)}.png'
    # Save the image
    cv2_im_processed = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'{drive_directory}{name_file}', cv2_im_processed)

    return name_file


##Upload image in imgbb

In [88]:
def upload_img(name_file):
    with open(f'{drive_directory}{name_file}', "rb") as file:
        url = "https://api.imgbb.com/1/upload"
        payload = {
            "key": KEY_IMGBB,
            "image": base64.b64encode(file.read()),
        }
        response = requests.post(url, payload)
        return (json.loads(response.text.encode('utf8')))

##HTML


In [13]:
def build_html(data_json):
    html = (""" 
    <center>
            <table border="0" cellpadding="0" cellspacing="0" width="100%" style="border-collapse:collapse;border:0;max-width:600px!important">
                <tr>
                    <td style="padding-top:9px;padding-left:18px;padding-bottom:9px;padding-right:18px">
                        <table border="0" cellspacing="0" width="100%" style="min-width:100%!important;background-color:#e2e2e2;border-collapse:collapse">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding:18px;color:#ffffff;font-family:Arial, Helvetica, sans-serif;font-size:14px;font-weight:normal;text-align:center;word-break:break-word;line-height:150%">
                                        <div style="text-align:center"><span style="color:#db6c2f"><span style="font-size:18px"><span style="font-family:source sans pro,helvetica neue,helvetica,arial,sans-serif">                                
                                        <strong>De: """ +data_json['name_sent'] + """</strong>
                                        <br><br>
                                        <strong>Para: """ +data_json['name'] + """</strong>
                                        </span></span>
                                            </span>
                                        </div>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td valign="top" style="padding-top:9px">
                        <table align="left" border="0" cellpadding="0" cellspacing="0" style="max-width:100%;min-width:100%;border-collapse:collapse" width="100%">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding-top:0;padding-right:18px;padding-bottom:9px;padding-left:18px;word-break:break-word;color:#202020;font-family:Arial;font-size:16px;line-height:150%;text-align:left">
                                        <div style="text-align:center"><span style="color:#808080"><span style="font-family:arial"><span style="font-size:14px">O time de Advanced Analytics informa que</span><br>
                                            <span style="font-size:15px"><strong>VOCÊ RECEBEU UM CARD DA GENTILEZA</strong></span></span>
                                            </span>
                                        </div>

                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td style="min-width:100%;padding:36px 18px 20px">
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" style="min-width:100%;border-collapse:collapse">
                            <tbody>
                                <tr>
                                    <td>
                                        <center>
                                            <img src=""" + data_json['card_url']+ """ alt=""" + data_json['card_url']+ """ width=100% height=100%>
                                        </center>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td style="min-width:100%;padding:36px 18px 20px">
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" style="min-width:100%;border-top:2px solid #e4e4e4;border-collapse:collapse">
                            <tbody>
                                <tr>
                                    <td>
                                        <span></span>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td valign="top" style="padding-top:9px">
                        <table align="left" border="0" cellpadding="0" cellspacing="0" style="max-width:100%;min-width:100%;border-collapse:collapse" width="100%">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding-top:0;padding-right:18px;padding-bottom:9px;padding-left:18px;word-break:break-word;color:#202020;font-family:Helvetica;font-size:16px;line-height:150%;text-align:left">
                                        <div style="text-align:center"><span style="font-size:14px"><span style="color:#696969"><em><span style="font-family:source sans pro,helvetica neue,helvetica,arial,sans-serif">Muito bom ser reconhecido, né?&nbsp;</span></em>
                                            </span>
                                            </span><br>
                                            <span style="font-size:14px"><span style="color:#696969"><em><span style="font-family:source sans pro,helvetica neue,helvetica,arial,sans-serif"><strong>Não esqueça de enviar outro para alguém que você também admira:</strong></span></em>
                                            </span>
                                            </span>
                                        </div>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td style="padding-top:0;padding-right:18px;padding-bottom:18px;padding-left:18px" valign="top" align="center">
                        <table border="0" cellpadding="0" cellspacing="0" style="border-collapse:separate!important;border:2px solid #1b365d;border-radius:2px">
                            <tbody>
                                <tr>
                                    <td align="center" valign="middle" style="font-family:Arial;font-size:14px;padding:10px">
                                        <a title="DISPARE ELOGIOS AGORA!" href="https://docs.google.com/forms/d/e/1FAIpQLSeh3G_V0G31YopLV69YJ8XOyTSQ7NKpcpPJ_GVYZNZ-ciX91w/viewform?utm_source=SulAm%C3%A9rica+Sa%C3%BAde&amp;utm_campaign=3ebdcde4f7-EMAIL_CAMPAIGN_2020_01_09_02_13_COPY_01&amp;utm_medium=email&amp;utm_term=0_c84c650c95-3ebdcde4f7-"
                                            style="font-weight:bold;letter-spacing:2px;line-height:100%;text-align:center;text-decoration:none;color:#1b365d;display:block" target="_blank" data-saferedirecturl="https://www.google.com/url?q=https://docs.google.com/forms/d/e/1FAIpQLSeh3G_V0G31YopLV69YJ8XOyTSQ7NKpcpPJ_GVYZNZ-ciX91w/viewform?utm_source%3DSulAm%25C3%25A9rica%2BSa%25C3%25BAde%26utm_campaign%3D3ebdcde4f7-EMAIL_CAMPAIGN_2020_01_09_02_13_COPY_01%26utm_medium%3Demail%26utm_term%3D0_c84c650c95-3ebdcde4f7-&amp;source=gmail&amp;ust=1601389843029000&amp;usg=AFQjCNGIP4P3Syu8FMnqUTK-E7QjY9SQaQ">DISPARE ELOGIOS AGORA!</a>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td valign="top" style="padding-top:9px">
                        <table align="left" border="0" cellpadding="0" cellspacing="0" style="max-width:100%;min-width:100%;border-collapse:collapse" width="100%">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding-top:0;padding-right:18px;padding-bottom:9px;padding-left:18px;word-break:break-word;color:#202020;font-family:Helvetica;font-size:16px;line-height:150%;text-align:left">
                                        <div style="text-align:center"><span style="font-size:14px"><font color="#696969" face="source sans pro, helvetica neue, helvetica, arial, sans-serif"><em>Acreditamos no poder dos elogios!&nbsp;</em><img goomoji="1f44a" data-goomoji="1f44a" style="margin:0 0.2ex;vertical-align:middle;max-height:24px" alt="👊" src="https://mail.google.com/mail/e/1f44a" data-image-whitelisted="" class="CToWUd">&nbsp;</font></span></div>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
                <tr>
                    <td valign="top" id="m_-1533314200010094230m_-1809350913193624200m_966440381877846507templateFooter" style="background:#1b365d none no-repeat center/cover;background-color:#1b365d;background-image:none;background-repeat:no-repeat;background-position:center;background-size:cover;border-top:0;border-bottom:0;padding-top:9px;padding-bottom:9px">
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" style="min-width:100%;border-collapse:collapse">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding-top:9px">
                                        <table align="left" border="0" cellpadding="0" cellspacing="0" style="max-width:100%;min-width:100%;border-collapse:collapse" width="100%">
                                            <tbody>
                                                <tr>
                                                    <td valign="top" style="padding:0px 18px 9px;color:#050202;word-break:break-word;font-family:Helvetica;font-size:12px;line-height:150%;text-align:center">
                                                        <a href="https://sulamerica.us20.list-manage.com/track/click?u=352dc06c43b28c8dfc93cf81e&amp;id=b789fbfa3d&amp;e=3c71f99bae&amp;utm_source=SulAm%C3%A9rica+Sa%C3%BAde&amp;utm_campaign=3ebdcde4f7-EMAIL_CAMPAIGN_2020_01_09_02_13_COPY_01&amp;utm_medium=email&amp;utm_term=0_c84c650c95-3ebdcde4f7-"
                                                            style="color:#656565;font-weight:normal;text-decoration:underline" target="_blank" data-saferedirecturl="https://www.google.com/url?q=https://sulamerica.us20.list-manage.com/track/click?u%3D352dc06c43b28c8dfc93cf81e%26id%3Db789fbfa3d%26e%3D3c71f99bae%26utm_source%3DSulAm%25C3%25A9rica%2BSa%25C3%25BAde%26utm_campaign%3D3ebdcde4f7-EMAIL_CAMPAIGN_2020_01_09_02_13_COPY_01%26utm_medium%3Demail%26utm_term%3D0_c84c650c95-3ebdcde4f7-&amp;source=gmail&amp;ust=1601389843029000&amp;usg=AFQjCNFjzengFkRZUobMElrNcGlJM6uwTA"><span style="color:#dc6b2f"><strong>Acesse o portal do #SULANALYTICSTEAM</strong></span></a><br>
                                                        <br>
                                                        <span style="color:#ffffff"><strong>Ajude o Office a melhorar a comunicação dos times de&nbsp; Analytics!</strong><br>
        Envie suas sugestões e ideias clicando&nbsp;</span><a href="mailto:data.anaytics@sulamerica.com.br" style="color:#656565;font-weight:normal;text-decoration:underline" target="_blank"><span style="color:#ffffff">AQUI!</span></a>
                                                    </td>
                                                </tr>
                                            </tbody>
                                        </table>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                        <table border="0" cellpadding="0" cellspacing="0" width="100%" style="min-width:100%;border-collapse:collapse">
                            <tbody>
                                <tr>
                                    <td valign="top" style="padding:9px">
                                        <table align="left" width="100%" border="0" cellpadding="0" cellspacing="0" style="min-width:100%;border-collapse:collapse">
                                            <tbody>
                                                <tr>
                                                    <td valign="top" style="padding-right:9px;padding-left:9px;padding-top:0;padding-bottom:0;text-align:center">
                                                        <img align="center" alt="" src="https://ci5.googleusercontent.com/proxy/rQL63l4tQ8PKCp19Oi8m33UfOOm1Ieqeo6Pgu3O43Y2QfTCXPhQtnfg-pkxtAgJ9a1FBE5H1wAwA1ICDf_LwmJ-QVu_O53vZjyceo2R8K9G_aLdi3OJU3iBV3RX4opD5kOixIP1gMilz-XJoUl0hwGivjDT7HQ=s0-d-e1-ft#https://mcusercontent.com/352dc06c43b28c8dfc93cf81e/images/5ebbf1d1-fd7c-4a25-9b53-b504657cbec4.png"
                                                            width="439.92" style="max-width:1700px;padding-bottom:0;display:inline!important;vertical-align:bottom;border:0;height:auto;outline:none;text-decoration:none" class="CToWUd">
                                                    </td>
                                                </tr>
                                            </tbody>
                                        </table>
                                    </td>
                                </tr>
                            </tbody>
                        </table>
                    </td>
                </tr>
            </table>
        </center>
     """)
    return html

#Data



In [89]:
data = gc.open('Ação_Card_da_Gentileza_On-line (respostas)').sheet1

In [90]:
df = pd.DataFrame(data.get_all_records())
df.rename(columns={'Carimbo de data/hora': 'datetime',
    '1 - Insira o NOME COMPLETO do PARCEIRO que você GOSTARIA DE ELOGIAR (DESTINATÁRIO):': 'name',
    '2 - Deixe aqui A MENSAGEM para o seu colega de trabalho (limite até 570 caracteres)': 'text',
    '3 - Escolha algum dos CARDS abaixo, para enviar para sua(seu) parceira(o):':'card'},inplace=True)
df=df[df["ENVIADO [y/n]"].str.lower()!='y']
df_error = df
df.shape

(4, 11)

In [91]:
df['num']=df['text'].str.len()
df.head(20)

,datetime,Endereço de e-mail,By,name,text,card,Quais sugestões daria para que essa ação fosse aperfeiçoada no futuro?,"E para finalizar, de 1 a 5, o quanto você acha importante que essa ação seja instalada dentro do time?","Caso queira registrar mais algum ponto, sinta-se à vontade!",Insira o seu NOME COMPLETO (REMETENTE):,ENVIADO [y/n],num
200,29/09/2020 15:11:45,alinebatista.mjv@sulamerica.com.br,Aline Batista,Denise Santos,"Denise, você chegou no momento de furação! heh...",GUERREIRA | Analytics Lovers!,,5,,,,124
201,30/09/2020 10:35:07,barbaralamego.keyrus@sulamerica.com.br,Barbara Lamego,Renan Ferreira Lima,"É um prazer trabalhar com você, sua dedicação ...",TEAM | Star JuntEs!,,5,,,,59
202,30/09/2020 11:47:30,juliana.silva@sulamerica.com.br,Juliana Silva,Bianca Paiva,"Bianca!!! Você é maravilhosa, coração enorme, ...",TEAM | Star JuntEs!,,5,,,,318
203,30/09/2020 12:08:58,juliana.silva@sulamerica.com.br,Juliana Silva,Queziane Marques,"Quezinha, querida!!! Parabéns pelo seu excelen...",GUERREIRA | Analytics ON!,Opções de carinhas nas mensagens rsrs,5,,,,289


In [92]:
df=df[df['text'].str.len()<=600]
df.head()

,datetime,Endereço de e-mail,By,name,text,card,Quais sugestões daria para que essa ação fosse aperfeiçoada no futuro?,"E para finalizar, de 1 a 5, o quanto você acha importante que essa ação seja instalada dentro do time?","Caso queira registrar mais algum ponto, sinta-se à vontade!",Insira o seu NOME COMPLETO (REMETENTE):,ENVIADO [y/n],num
200,29/09/2020 15:11:45,alinebatista.mjv@sulamerica.com.br,Aline Batista,Denise Santos,"Denise, você chegou no momento de furação! heh...",GUERREIRA | Analytics Lovers!,,5,,,,124
201,30/09/2020 10:35:07,barbaralamego.keyrus@sulamerica.com.br,Barbara Lamego,Renan Ferreira Lima,"É um prazer trabalhar com você, sua dedicação ...",TEAM | Star JuntEs!,,5,,,,59
202,30/09/2020 11:47:30,juliana.silva@sulamerica.com.br,Juliana Silva,Bianca Paiva,"Bianca!!! Você é maravilhosa, coração enorme, ...",TEAM | Star JuntEs!,,5,,,,318
203,30/09/2020 12:08:58,juliana.silva@sulamerica.com.br,Juliana Silva,Queziane Marques,"Quezinha, querida!!! Parabéns pelo seu excelen...",GUERREIRA | Analytics ON!,Opções de carinhas nas mensagens rsrs,5,,,,289


In [93]:
df.shape

(4, 12)

In [94]:
df = df[
   [    'datetime',    'By',   'name',   'text',    'card'    ]
   ]
df['card_file'] = df['card'].str.replace('|', '').str.replace('!', '').str.replace('  ', ' ').str.replace(' ', '_').str.lower()
df['text'] = df['text'].str.replace('\n', ' ')
df['name'] = df['name'].str.split('(').str[0]
df=df[df['By'].str.len()>0]
display(df.head())

,datetime,By,name,text,card,card_file
200,29/09/2020 15:11:45,Aline Batista,Denise Santos,"Denise, você chegou no momento de furação! heh...",GUERREIRA | Analytics Lovers!,guerreira_analytics_lovers
201,30/09/2020 10:35:07,Barbara Lamego,Renan Ferreira Lima,"É um prazer trabalhar com você, sua dedicação ...",TEAM | Star JuntEs!,team_star_juntes
202,30/09/2020 11:47:30,Juliana Silva,Bianca Paiva,"Bianca!!! Você é maravilhosa, coração enorme, ...",TEAM | Star JuntEs!,team_star_juntes
203,30/09/2020 12:08:58,Juliana Silva,Queziane Marques,"Quezinha, querida!!! Parabéns pelo seu excelen...",GUERREIRA | Analytics ON!,guerreira_analytics_on


In [95]:
df['text'].str.len().describe()

count      4.000000
mean     197.500000
std      125.800106
min       59.000000
25%      107.750000
50%      206.500000
75%      296.250000
max      318.000000
Name: text, dtype: float64

#Send

In [96]:
df_edit = df
emails_list = [        
        'brunonascimento.leega@sulamerica.com.br',
        'advanced.analytics@sulamerica.com.br',
        # 'alinebatista.mjv@sulamerica.com.br'
    ]
for index, row in df_edit.iterrows():    
    text = str(row['text']).replace('Ass','\nAss')
    response_img = upload_img(create_img(row['card_file'],text))
    data_json ={
        'name':row['name'],   
        'name_sent':row['By'],     
        'card_url':response_img['data']['url']
    }
    mensage = build_html(data_json)
    print(data_json)
    sendemail(
        from_addr='bruno.nascimento@leega.com.br',
        to_addr_list=list(dict.fromkeys(emails_list)),
        subject=(f'{row["name"]} Você recebeu um CARD DA GENTILEZA On-line! 📢🌟'),
        message=mensage
        )
    


{'name': 'Denise Santos', 'name_sent': 'Aline Batista', 'card_url': 'https://i.ibb.co/S6vMvc9/0d6b991ace6b.png'}
202
b''
Server: nginx
Date: Wed, 07 Oct 2020 14:09:34 GMT
Content-Length: 0
Connection: close
X-Message-Id: H-9wK5JcT5aF2jHZQ5O_qQ
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html


{'name': 'Renan Ferreira Lima', 'name_sent': 'Barbara Lamego', 'card_url': 'https://i.ibb.co/SKX3Fzp/331379b5d6d1.png'}
202
b''
Server: nginx
Date: Wed, 07 Oct 2020 14:09:35 GMT
Content-Length: 0
Connection: close
X-Message-Id: 5O4mFDN6SfmwwW2HMF_DGA
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Re

In [97]:
print(f'{df_edit.shape[0]} emails were sent.')

4 emails were sent.


#Error table

In [98]:
df_error[(df_error['text'].str.len()>600) | (df_error['By'].str.len()<=0)]

,datetime,Endereço de e-mail,By,name,text,card,Quais sugestões daria para que essa ação fosse aperfeiçoada no futuro?,"E para finalizar, de 1 a 5, o quanto você acha importante que essa ação seja instalada dentro do time?","Caso queira registrar mais algum ponto, sinta-se à vontade!",Insira o seu NOME COMPLETO (REMETENTE):,ENVIADO [y/n],num


#Clear drive

In [99]:
arr = os.listdir(drive_directory)
for to_del in arr:
    if to_del.startswith("output_"):
        print(f'Remove: {to_del}')
        os.remove(f'{drive_directory}{to_del}')

Remove: output_0135_EQZGSPFjOG.png
Remove: output_0229_QqHNzMPAcE.png
Remove: output_0019_OGhdRiyQdV.png
Remove: output_0318_BtJCeFBDSp.png
Remove: output_0431_LieGWoTpRc.png
Remove: output_0866_lzqEomPpjL.png
Remove: output_0643_qcnhybOMta.png
